In [127]:
import json
from functools import partial
from inspect import getsource
from math import log
from sys import argv
from bisect import bisect_left

if __name__ == '__main__':
    map_in_name, script_name = '../cases/case1.json', 'test.py'
    nf = '../input/case1_in.json'
    nf = open(nf, 'r')
    nf = json.load(nf)
    keys, values = sorted(nf[0]), sorted(nf[1])
    
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()

    mmm = sorted(list(origin_mapping.items()),key=lambda x: x[0])

    k,v = zip(*mmm)
    mm = sorted(zip(v,[ i for i in range(len(k)) ]),key=lambda x: x[0])
    v, idx = zip(*mm)
    idx = list(idx)
    
    mark = []
    temp = index, idx_start, count = 0, 0, 0 
    flag = False
    for i in range(1,len(k)):
        if i != idx[i]:
            if flag == False:
                flag = True
                index = i
                idx_start = idx[i]
            count +=1
        else:
            flag = False
            mark.append(temp)
    print(len(str(mark)))
    document = """
import json
import hashlib
from sys import argv

_m = {}
nf, of = argv[1], argv[2]
nf = open(nf, 'r')
nf = json.load(nf)
keys, values = sorted(nf[0]), sorted(nf[1])

nm = {{}}
for i in range(len(keys)):
    nm[keys[_m[i]]] = values[i]
of = open(of, 'w')
of.write(json.dumps(nm))
of.close()
    """.format(str(mark))
    script = open(script_name, 'w')
    script.write(document)
    script.close()

9603
